Installing required libraries for stepfunctions

In [ ]:
 import sys
 !{sys.executable} -m pip install --upgrade pip
 !{sys.executable} -m pip install -qU awscli boto3 "sagemaker>=2.0.0"
 !{sys.executable} -m pip install -qU "stepfunctions>=2.0.0"
 !{sys.executable} -m pip show sagemaker stepfunctions

# 1. Setting up notebook with parameters and libraries

In [ ]:
# importing Python libraies
import stepfunctions
import logging

from stepfunctions.steps import *
from stepfunctions.workflow import Workflow
from stepfunctions import steps
from stepfunctions.inputs import ExecutionInput
from sagemaker.processing import Processor,ProcessingInput, ProcessingOutput
import calendar
import time
import sagemaker
from sagemaker.inputs import TrainingInput
import boto3
from sagemaker.network import NetworkConfig

stepfunctions.set_stream_logger(level=logging.INFO)

### Defining paramters

This needs to be changed if we are taking it to different environment

In [ ]:
v_workflow_execution_role = "arn:aws:iam::525102048888:role/poc-sagemaker-step-functi-MachineLearningWorkflowE-1XFI2UPRXFTXE" # Step function IAM role ARN
v_preprocessing_iam_role = "arn:aws:iam::525102048888:role/service-role/AmazonSageMaker-ExecutionRole-20191105T125227" # IAM role for preprocessing container
v_preprocessing_instance_type = "ml.m5.xlarge" # Instance type for preprocessing container it changes as per workload
v_s3_input_bucket = "wi-cred-datalake-dev-raw" # S3 bucket for input and output data
v_prefix_for_input_data = "data/output/offlease/lr/baselineinp/"  # Prefix where data is stored
v_region = 'us-east-1' # AWS region
sec_groups = ["sg-044e0e7ce4f5721c0"]
subnets = ["subnet-0cf0e3f46326aa259",
           "subnet-0156b7f5500cf0b78",
           "subnet-032420199163cff9b"]
config_bucket = "wi-cred-datalake-dev-s3-mlops-config"

## 2. Defining preprocessing jobs

In [ ]:
# Defining environment config for baseline jobs
environment = {
          "dataset_format": "{\"csv\":{\"header\": true,\"output_columns_position\": \"START\"}}",
          "dataset_source": "/opt/ml/processing/input/baseline_dataset_input",
          "output_path": "/opt/ml/processing/output",
          "publish_cloudwatch_metrics": "Disabled"
        }

In [ ]:
# Here we are creating baseline preprocesor
baseline_processor = Processor(image_uri='156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer',
                     role=v_preprocessing_iam_role,
                     instance_count=1,
                     instance_type=v_preprocessing_instance_type,
                     network_config = NetworkConfig(security_group_ids = sec_groups, subnets = subnets),
                     env=environment)

In [ ]:
input_data = "s3://{}/{}".format(v_s3_input_bucket,v_prefix_for_input_data)
inputs = [
    ProcessingInput(
        source=input_data, destination="/opt/ml/processing/input/baseline_dataset_input", input_name="input_data"
    )
]

outputs = [
    ProcessingOutput(
        source="/opt/ml/processing/output",
        destination="s3://{}/{}".format(config_bucket,"custommonitor/"),
        output_name="tr_data",
    )
]
gmt = time.gmtime()
ts = calendar.timegm(gmt)
baseline_name = "baseline-{}".format(ts)

In [ ]:
print("s3://{}/{}".format(config_bucket,"customonitor"))


In [ ]:
baseline_preprocessing_step = steps.ProcessingStep(
    state_id='Baseline', 
    processor=baseline_processor,
    job_name=baseline_name, 
    inputs=inputs, 
    kms_key_id='3084dc48-1a82-435b-8a8d-8001f8890c08',
    outputs=outputs, 
    experiment_config=None, 
    wait_for_completion=True
)

## 3. Step Function

In [ ]:
# First we chain the start pass state,preprocessing_step,
basic_path=Chain([baseline_preprocessing_step])

In [ ]:
import time
print(time.localtime())

In [ ]:
# Next, we define the workflow
import uuid
basic_workflow = Workflow(
    name="ds-mlops-dev-baseline-job".format(
    uuid.uuid1().hex
) ,
    definition=basic_path,
    role=v_workflow_execution_role
)

#Render the workflow
basic_workflow.render_graph()

## 3.1 Create the workflow on AWS Step Functions

Create the workflow in AWS Step Functions with [create](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.create).

In [ ]:
basic_workflow.create()

In [ ]:
basic_workflow.update(definition=basic_workflow.definition,role=basic_workflow.role)

In [ ]:
basic_workflow_execution = basic_workflow.execute(
    inputs={
    }
)

## 3.2 Review the execution progress

Render workflow progress with the [render_progress](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Execution.render_progress).

This generates a snapshot of the current state of your workflow as it executes. This is a static image. Run the cell again to check progress. 

In [ ]:
basic_workflow_execution.render_progress()

## 4 Downloading generated report on notebook

In [ ]:
# Downloading contraint file for evaluation
!aws s3 cp s3://$config_bucket/custommonitor/constraints.json .

In [ ]:
# Dowloading stats file for evaluation
!aws s3 cp s3://$config_bucket/custommonitor/statistics.json .

We can perform analysis on this file and put it on git hub

**Note :**
In order to refer it for modified location in monitoring schedule lambda function we need to changes the baseline stats location to prefix custom_monitoring/ from monitoring
